In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
import numpy as np
from cp2k_file_utils import read_energy
from pao_file_utils import parse_pao_file, write_pao_file
from pao_tfn_trainer import loss_function, ortho

In [8]:
frame_dir = "2H2O_MD/frame_0000"
kinds, atom2kind, coords, xblocks = parse_pao_file(frame_dir + "/2H2O_pao44-1_0.pao")

for i in range(11):
    eps = i * 0.01
    losses = []
    perturbed_xblocks = []
    for xblock in xblocks:
        perturbed_xblock = torch.from_numpy(xblock + eps*np.random.rand(*xblock.shape))
        perturbed_xblock = ortho(perturbed_xblock)
        perturbed_xblocks.append(perturbed_xblock)
        losses.append(loss_function(perturbed_xblock, torch.from_numpy(xblock)))
    loss = np.mean(losses)

    write_pao_file(frame_dir + "/inferred.pao", kinds, atom2kind, coords, perturbed_xblocks)
    ! cd $frame_dir; /opt/cp2k/exe/local/cp2k.ssmp -i 2H2O_pao44_inferred.inp > 2H2O_pao44_inferred.out
    energy = read_energy(frame_dir+"/2H2O_pao44_inferred.out")
    
    print("energy: {}  loss: {:g}  eps: {:g}".format(energy, loss, eps))

energy: -34.449936138968745  loss: 4.95281e-33  eps: 0
energy: -34.44608609060372  loss: 8.81627e-06  eps: 0.01
energy: -34.436347034363656  loss: 3.36408e-05  eps: 0.02
energy: -34.416961415462694  loss: 6.80836e-05  eps: 0.03
energy: -34.38718169357887  loss: 0.000139983  eps: 0.04
energy: -34.35367771716608  loss: 0.000225817  eps: 0.05
energy: -34.28329230009149  loss: 0.000386685  eps: 0.06
energy: -34.2424488152231  loss: 0.00052577  eps: 0.07
energy: -34.1890870136628  loss: 0.000584067  eps: 0.08
energy: -34.135131242913396  loss: 0.00069215  eps: 0.09
energy: -34.046954564224905  loss: 0.000914383  eps: 0.1
